>**ENFOQUE: "CRIPTOGRAFÍA"**

*Tipo de arbol: "Tree Merkle"*

>[1] **¿Qué es un arbol de merkle?**

    "Un árbol de Merkle es una estructura de datos en forma de árbol donde cada nodo hoja es un hash criptográfico que representa un bloque de datos, y cada nodo no hoja es el hash de los hijos inmediatos. Esta estructura se utiliza en criptografía para verificar la integridad de datos y asegurar que no ha habido cambios no autorizados en un conjunto de datos" (Merkle, 1987).

>[2] **Problema a resolver:**

**Pruebas de conocimiento cero ( Zero Knowledge Proof)**

- **¿Qué es la prueba de conocimiento cero?**

        En criptografía, un protocolo de conocimiento cero o prueba de conocimiento nulo, también conocidas por las siglas ZKP, es un  protocolo criptográfico que establece un metodo para que una de las partes prueba a otra que una declaración (generalmente matemática) es  cierta, sin revelar nada más que la veracidad de la declaración.     

- **¿Comó se usa el arbol de merkle en la prueba de conocimiento cero?**

        El árbol de Merkle se utiliza en la prueba de conocimiento cero (ZKP) para permitir que un participante demuestre que conoce un valor secreto sin revelarlo al otro participante. El participante que conoce el valor secreto construye un árbol de Merkle que incluye el valor secreto como una hoja del árbol, y luego genera una prueba que consiste en una lista de hashes de hermanos en el árbol de Merkle. El otro participante puede verificar la prueba comparando los hashes de los hermano proporcionados con los hashes en el árbol de Merkle que tiene. Esto permite que el participante que no conoce el valor secreto verifique que el participante que conoce el valor secreto tiene conocimiento del valor sin revelarlo.
                


    

>[3] **Codigo - Explicación - Ejemplo**

- **Codigo - Ejemplo**

In [5]:
import hashlib
 
class MerkleTree:
    """
    Clase que representa un árbol de Merkle.

    Un árbol de Merkle es un árbol binario que se utiliza para verificar la integridad de datos.
    En un árbol de Merkle, cada hoja representa un fragmento de datos y cada nodo interno es
    el hash de sus dos hijos

    Args:
        leaves (list): Lista de hashes de las hojas del árbol
        levels (list): Lista de niveles del árbol, donde cada nivel es una lista de hashes
    """

    def __init__(self, data):
        """
        Inicializa un nuevo árbol de Merkle con los datos proporcionados

        Args:
            data (list): Lista de datos a incluir en el árbol
        """

        # Calcula el hash de cada dato en la lista y lo agrega como hoja del árbol
        self.leaves = [hashlib.sha256(d.encode()).hexdigest() for d in data]

        # Calcula los niveles del árbol, comenzando por las hojas y subiendo hacia la raíz
        self.levels = [self.leaves]
        while len(self.levels[-1]) > 1:
            level = self.levels[-1]
            next_level = [self.hash_pairs(level[i], level[i+1]) for i in range(0, len(level), 2)]
            self.levels.append(next_level)

    def hash_pairs(self, a, b):
        """
        Combina dos hashes en un nuevo hash

        Args:
            a (str): Primer hash a combinar
            b (str): Segundo hash a combinar

        Returns:
            str: El nuevo hash resultante de combinar los dos hashes de entrada
        """
        return hashlib.sha256((a + b).encode()).hexdigest()

    def root(self):
        """
        Retorna el hash de la raíz del árbol

        Returns:
            str: El hash de la raíz del árbol
        """
        return self.levels[-1][0]

class ZeroKnowledgeProof:
    """
    Clase para generar y verificar pruebas de conocimiento cero utilizando un árbol de Merkle
    """
    def __init__(self, secret):
        """
        Inicializa un objeto ZeroKnowledgeProof

        Parameters:
            secret (str): El secreto a utilizar para generar la prueba

        Returns:
            None
        """
        self.secret = secret

    def generate_proof(self, leaf_index):
        """
        Genera una prueba de conocimiento cero para una hoja específica del árbol

        Parameters:
            leaf_index (int): El índice de la hoja para la cual se quiere generar la prueba

        Returns:
            list: Una lista con los hermanos de la hoja en el árbol, utilizados para generar la prueba
        """
        proof = []
        tree = MerkleTree([self.secret])
        level_index = len(tree.levels) - 2
        sibling_index = leaf_index ^ 1
        for level in reversed(tree.levels[:-1]):
            sibling = level[sibling_index]
            proof.append(sibling)
            sibling_index //= 2
        return proof

    def verify_proof(self, leaf_value, leaf_index, proof, root):
        """
        Verifica una prueba de conocimiento cero para una hoja específica del árbol

        Parameters:
            leaf_value (str): El valor de la hoja que se quiere verificar
            leaf_index (int): El índice de la hoja que se quiere verificar
            proof (list): Una lista con los hermanos de la hoja en el árbol, utilizados para generar la prueba
            root (str): La raíz del árbol.

        Returns:
            bool: True si la prueba es válida, False en caso contrario
        """
        value = leaf_value
        for i, sibling in enumerate(proof):
            if leaf_index % 2 == 0:
                value = hashlib.sha256((value + sibling).encode()).hexdigest()
            else:
                value = hashlib.sha256((sibling + value).encode()).hexdigest()
            leaf_index //= 2
        return value == root
    
# Ejemplo de uso:
secret = "TeAmoJesus"
proof_index = 0  # Prueba para la hoja 0 (el secreto)

proof = ZeroKnowledgeProof(secret).generate_proof(proof_index)

tree = MerkleTree([secret])
root = tree.root()

is_verified = ZeroKnowledgeProof(secret).verify_proof(tree.leaves[proof_index], proof_index, proof, root)

print("La prueba es válida: ", is_verified)

La prueba es válida:  True


- **Explicación**

    *Antes de seguir con la explicación es indispensable comentar que este código fue realizado por ChatGPT*

    En este caso yo tengo un secreto que es "secret = TeAmoJesus", y quiere demostrarle a Jesus que conocel el secreto, sin revelar este. 

    Primero construimos un arbol de merkle:

        >secret = "TeAmoJesus"
        >tree = MerkleTree([secret])
    
    El árbol de Merkle tiene solo una hoja, que corresponde a mi secreto. La raíz del árbol es la suma de los hashes de las hojas:

        >root = tree.root()
        
    - *" **¿Qué es un hash?:** un hash es una función criptográfica que toma una cadena de datos como entrada y produce una cadena de longitud fija que representa el "hash" de esos datos. El hash se usa para verificar la integridad de los datos en el árbol, ya que cualquier cambio en los datos producirá un hash diferente. Los hashes se utilizan en cada nodo del árbol para asegurar que los datos que se encuentran en las hojas son los mismos que los datos en la raíz del árbol."*
    

    Seguidamente, genero una prueba de conocimiento cero para la hoja del árbol que contiene el secreto. En este caso, la hoja es la única hoja del árbol, por lo tanto, tiene indice "0"

        >proof_index = 0
        >proof = ZeroKnowledgeProof(secret).
        generate_proof(proof_index)
    
    La prueba que se genero anteriormente es una lista de hashes de hermanos que le permiten a Jesus verificar que conoce el valor de mi secreto en la hoja correspondiente.

    Para finalizar, se envía la prueba de conocimiento cero y la raíz del arbol a Jesus. Este utiliza estos dos ultimos para verificar que yo conozco el valor de la hoja correspondiente a mi secreto, sin revelar el secreto.

        >is_verified = ZeroKnowledgeProof(secret).
        verify_proof(tree.leaves[proof_index],
        proof_index, proof, root)
    
    Esto es lo que imprime el valor de la prueba, puede ser "True" o "False".

        >print("La prueba es válida: ", is_verified)

    Si la prueba es "True", significa que Jesus sabe que yo sé cual es el valor de la hoja correspondiente del árbol de merkle, sin conocer o revelar el secreto mismo.
     



>[4] **¿Por qué escogimos este problema?**

- Escogimos este problema porque es un ejemplo práctico del uso de la estructura de datos del árbol de Merkle en la criptografía. La prueba de conocimiento cero es una técnica criptográfica que permite a una persona demostrar que posee cierta información sin revelarla, y el árbol de Merkle es utilizado para crear pruebas de conocimiento cero que sean verificables de forma eficiente. 


>[5] **¿Cuál fue el interés en este tema?**

- Este fue de nuestro interés porque uno de los integrantes de nuestro grupo esta interesado en la ciberseguridad.

>[6]  **Relación ciberseguridad - Árbol de Merkle**

- El árbol de Merkle se utiliza en ciberseguridad para verificar la integridad de datos y asegurar la autenticidad de la información. Por ejemplo, en la tecnología blockchain, cada bloque en la cadena de bloques incluye un hash que representa la transacción anterior y se utiliza para verificar que el bloque no haya sido alterado. También se puede utilizar en sistemas de autenticación, como la autenticación de dos factores, para verificar la autenticidad de un usuario sin revelar información confidencial.

**Referencias:**

[1] Merkle, R.C., "A digital signature based on a conventional encryption function," Advances in Cryptology—CRYPTO '87, Springer Berlin Heidelberg, 369-378, 1988.

[2] Antonopoulos, A. M. (2014). Mastering Bitcoin: Unlocking Digital Cryptocurrencies. O'Reilly Media, Inc.

[3] B. Schneier, "Applied Cryptography: Protocols, Algorithms, and Source Code in C," John Wiley & Sons, Inc., 1996.

[4] OpenAI - ChatGpt